In [1]:
#Planetary Astrophysics Homework 3
#Tyler Nelson

import numpy as np
import scipy as sci
import math
import matplotlib.pyplot as plt
from functools import partial
from numpy import pi
import mpmath
from scipy import interpolate
from scipy import integrate
mpmath.mp.prec = 50

In [2]:
h = 6.6e-27
c = 3e10
kB = 1.4e-16
sigSB = 5.7e-5
aSB = 7.6e-15
AU = 1.5e13
rSun = 7e10
LSun = 3.8e33
mSun = 2e33
G = 6.7e-8
TSunEff = (LSun/(4*pi*rSun**2*sigSB))**(1/4)
def trapInt(f, a, b, dx):
    return dx/2*(f(a)+f(b)+ 2*sum(np.vectorize(f)(np.linspace(a+dx,b-dx,int((b-a)/dx)))))
def bb_nu(T, nu):
    return (2*h*nu**3/c**2)/(mpmath.exp(h*nu/(kB*T))-1)
def bb_l(T, l):
    return (2*h*c**2/l**5)/(mpmath.exp(h*c/(l*kB*T))-1)


In [3]:
def bbSpecL2(T, r, spacingInfo, scale=0):
    l = np.linspace(*spacingInfo)
    if scale == 0:
        return l, (2*pi*r)**2*np.vectorize(partial(bb_l, T))(l)
    return l, (pi*r/scale)**2*np.vectorize(partial(bb_l, T))(l)

def bbSpecNu2(T, r, spacingInfo, scale=0):
    l = np.linspace(*spacingInfo)
    if scale == 0:
        return l, (2*pi*r)**2*np.vectorize(partial(bb_nu, T))(l)
    return l, (pi*r/scale)**2*np.vectorize(partial(bb_nu, T))(l)

In [4]:
tStar = 8590
rStar = 1.9*rSun
dist = 10*AU
mStar = 1.92*mSun

In [37]:
fig = plt.figure()
sub1 = fig.add_subplot(211)
sub2 = fig.add_subplot(212)
x, y = bbSpecNu2(tStar, rStar,[3e13, 3e17, 1e5], scale =dist)
sub1.plot(c/x*1e4,y*1e23,'r', label="10 AU")
sub1.set_xlabel("$\\lambda (\\mu m)$")
sub1.set_ylabel("$f_{\\nu} (Jy)$")
sub1.set_title("Flux Calibrated Blackbody Specs for Fomalhaut")
x, y = bbSpecNu2(tStar, rStar, [3e13, 3e17, 1e5],scale=130*AU)
sub2.plot(c/x*1e4,y*1e23, label="130 AU")
sub2.set_xlabel("$\\lambda (\\mu m)$")
sub2.set_ylabel("$f_{\\nu} (Jy)$")
plt.tight_layout()
sub1.legend()
sub2.legend()
plt.show()

In [5]:
#C:\Users\Tyler\Desktop\suvSil_21
file = "suvSil_21"
path = r"C:\Users\Tyler\Desktop\{0}\{0}".format(file)

t1 = np.genfromtxt(path,delimiter=" ", skip_header=2448, usecols=(0,1),max_rows=(2689-2448))
vsmall = interpolate.interp1d(c/(t1[:,0]*1e-4),t1[:,1], kind="cubic")
t2 = np.genfromtxt(path,delimiter=" ", skip_header=3668, usecols=(0,1),max_rows=(3909-3668))
small = interpolate.interp1d(c/(t2[:,0]*1e-4),t2[:,1], kind="cubic")
t3 = np.genfromtxt(path,delimiter=" ", skip_header=4888, usecols=(0,1),max_rows=(5129-4888))
medium = interpolate.interp1d(c/(t3[:,0]*1e-4),t3[:,1], kind="cubic")

In [393]:

#janksy are in frequency not wavelength 


In [6]:
def powerAbs(Rg, Rs, a, Tstar, bbFunc, QFunc, start, stop, step):
    #return (Rg*Rs/a)**2*pi**2*trapInt(lambda l: bbFunc(Tstar,l)*QFunc(l),start,stop,step)
    return (Rg*Rs*pi/a)**2*integrate.quad(lambda x : bbFunc(Tstar, x)*QFunc(x), start,stop)[0]
def sb(R, L):
    return (L/(4*pi*R**2*sigSB))**(1/4)

In [7]:

sizeList = [0.1e-4,1e-4,10e-4,1e-1]
functionList = [vsmall, small, medium, lambda x : 1]


def glueFunction(sizeLst, rStr, dst, tStr, bbFunc, bbFuncSED, QFuncList, integrationInfo, sedInfo):
    length = len(sizeLst)
    rList1 = [0]*length #temp
    rList2 = [0]*length #sed
    PinList = [0]*length #p abs
    PoutList = [0]*length #p emm
    i = 0 #bbSpecL2(T, r, distInfo, scale=0)
    while i < length-1:
        PinList[i] = powerAbs(sizeLst[i], rStr, dst, tStr, bbFunc, QFuncList[i], *integrationInfo)
        rList1[i] = sb(sizeLst[i], PinList[i])
        rList2[i] = bbFuncSED(rList1[i], sizeLst[i], sedInfo)[1]
        PoutList[i] = (2*pi*sizeList[i])**2*integrate.quad(lambda l : bbFunc(rList1[i], l)*QFuncList[i](l),*integrationInfo[:2])[0]
        i+=1
    PinList[-1] = powerAbs(sizeLst[-1], rStr, dst, tStr, bbFunc, QFuncList[-1], *integrationInfo)
    rList1[-1] = sb(sizeLst[-1], PinList[-1])
    PoutList[-1] = (2*pi*sizeList[-1])**2*integrate.quad(lambda l : bbFunc(rList1[-1], l)*QFuncList[-1](l),*integrationInfo[:2])[0]
    horizontalRange, rList2[-1] = bbFuncSED(rList1[-1], sizeLst[-1],sedInfo)
    return rList1, rList2, horizontalRange, PinList, PoutList

In [8]:
def sedModFunc(sedList, collectingRadius, dist):#also assuming sed not in jankys
    returnList = [0]*len(sedList)
    for i in range(len(sedList)):
        returnList[i] = sedList[i]*1e23*(collectingRadius/(2*dist))**2
    return returnList

In [9]:
def sedPlotter(sed,xList, tList, sizeList, dist, fig, dCount=[], mode="nu"):
    l = len(sed)
    count = 0
    multiplier = [1]*l
    if(len(dCount)):
        multiplier = dCount
    while count < l:
        sub1 = fig.add_subplot(l*100+10+count+1)
        if mode == "nu":
            sub1.plot(c/xList*1e4, sed[count]*multiplier[count], label="%0.0f K"%tList[count])
            
        else:
            sub1.plot(xList*1e4, sed[count]*multiplier[count], label = "%0.0f K"%tList[count])
        sub1.set_title("{0} micron grains at {1} AU".format(sizeList[count]*1e4, dist/AU))
        sub1.set_xlabel("$\\lambda(\\mu m)$")
        sub1.set_ylabel("$f_{\\nu}(Jy)$")
        plt.legend()
        count+=1
    return fig


def dustCounterForMicrons(xList, wavelength, flux_jy, sedlist):
    l = len(sedlist)
    dCount = [0]*l
    i, j = 0, 0
    while xList[i] < wavelength:
        i+=1
    while j < l:
        dCount[j] = flux_jy/sedlist[j][i]
        j+=1
    return dCount

In [10]:
#pulling out all the calculations

TList1, SEDList1, xList1, Pin1, Pout1 = glueFunction(sizeList, rStar, dist, tStar, bb_nu, bbSpecNu2, functionList, [3e12,3e16,3e12], [3e12, 3e16, 1e5])
TList2, SEDList2, xList2, Pin2, Pout2 = glueFunction(sizeList, rStar, 130*AU, tStar, bb_nu, bbSpecNu2, functionList, [3e11,3e16,3e12], [3e11, 3e16, 1e5])

C:\Anaconda2\envs\python3\lib\site-packages\scipy\integrate\quadpack.py:357: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  warnings.warn(msg, IntegrationWarning)


In [11]:
SEDList1_m = sedModFunc(SEDList1, 0.5e2, 7.7*3.08e18)
SEDList2_m = sedModFunc(SEDList2, 0.5e2, 7.7*3.08e18)


In [14]:
#Su et al 2013 dust estimate 35 microns, 1 Jy, 
#about 1 Jy at 400 microns Ricci et al 2012

dCount1 = dustCounterForMicrons(c/xList1*1e4, 35, 1, SEDList1_m)
dCount2 = dustCounterForMicrons(c/xList2*1e4, 400, 1, SEDList2_m)

In [535]:


fig = plt.figure()
sedPlotter(SEDList1_m, xList1, TList1, sizeList, dist,fig)
plt.tight_layout()
plt.show()

In [573]:
fig = plt.figure()
sedPlotter(SEDList1_m, xList1, TList1, sizeList,dist,fig, dCount = dCount1)
plt.tight_layout()
plt.show()

In [574]:
fig = plt.figure()
sedPlotter(SEDList2_m, xList2, TList2, sizeList, 130*AU,fig)
plt.tight_layout()
plt.show()

In [584]:
fig = plt.figure()
sedPlotter(SEDList2_m, xList2, TList2, sizeList,130*AU,fig, dCount = dCount2)
plt.tight_layout()
plt.show()

In [15]:
def printerFunction(slist, dist, pinList, tList, dlist, density, poutList):
    print("grain size (microns) | Dist (AU) | P_in (ergs) | T_eff (K) | Number to get equivalent flux | Dust Mass (g)| P_out (ergs)")
    count = 0
    l = len(slist)
    while count < l:
        print("%20.1f | %9.0f | %11.2g | %9.0f | %29.3g | %13.3g | %10.2g"%(slist[count]*1e4, dist/AU, pinList[count],tList[count],dlist[count], density*4/3*pi*slist[count]**3*dlist[count], poutList[count]))
        count+=1

In [16]:
printerFunction(sizeList, dist, Pin1, TList1, dCount1, 2, Pout1)

grain size (microns) | Dist (AU) | P_in (ergs) | T_eff (K) | Number to get equivalent flux | Dust Mass (g)| P_out (ergs)
                 0.1 |        10 |     1.7e-05 |       125 |                      1.22e+31 |      1.02e+17 |    3.7e-07
                 1.0 |        10 |      0.0072 |       178 |                      6.99e+28 |      5.86e+17 |     0.0028
                10.0 |        10 |        0.75 |       180 |                      6.86e+26 |      5.75e+18 |        1.1
              1000.0 |        10 |     8.1e+03 |       184 |                      6.68e+22 |      5.59e+20 |    8.5e+03


In [17]:
printerFunction(sizeList, 130*AU, Pin2, TList2, dCount2, 2, Pout2)

grain size (microns) | Dist (AU) | P_in (ergs) | T_eff (K) | Number to get equivalent flux | Dust Mass (g)| P_out (ergs)
                 0.1 |       130 |       1e-07 |        35 |                      2.91e+33 |      2.44e+19 |    7.7e-22
                 1.0 |       130 |     4.2e-05 |        49 |                      1.91e+31 |       1.6e+20 |    1.7e-06
                10.0 |       130 |      0.0045 |        50 |                      1.88e+29 |      1.58e+21 |     0.0049
              1000.0 |       130 |          48 |        51 |                      1.84e+25 |      1.54e+23 |         51


In [18]:
#using the absorption formulation of PR drag 
def prrp(dist, mStar, pin, sizeList):#short for poynting-robertson radiation pressure, its a palindrome
    #unit of force is dyne
    print("grain size (microns) | Dist (AU) | Poynting-Robertson Drag (dyne) | Radiation Pressure (bayre)")
    count = 0
    l = len(pin)
    prList = [0]*l
    rpList = [0]*l
    while count < l:
        prList[count] = pin[count]/c**2*(G*mStar/dist)
        rpList[count] = pin[count]/c
        print("%20.1f | %9.0f | %30.2g | %26.2g"%(sizeList[count]*1e4, dist/AU, pin[count]/c**2*(G*mStar/dist), pin[count]/c))
        count+=1
    return prList, rpList

In [19]:
pr1, rp1 = prrp(dist, mStar, Pin1, sizeList)

grain size (microns) | Dist (AU) | Poynting-Robertson Drag (dyne) | Radiation Pressure (bayre)
                 0.1 |        10 |                        3.3e-14 |                    5.8e-16
                 1.0 |        10 |                        1.4e-11 |                    2.4e-13
                10.0 |        10 |                        1.4e-09 |                    2.5e-11
              1000.0 |        10 |                        1.6e-05 |                    2.7e-07


In [20]:
pr2, rp2 = prrp(130*AU, mStar, Pin2, sizeList)

grain size (microns) | Dist (AU) | Poynting-Robertson Drag (dyne) | Radiation Pressure (bayre)
                 0.1 |       130 |                        1.5e-17 |                    3.4e-18
                 1.0 |       130 |                        6.2e-15 |                    1.4e-15
                10.0 |       130 |                        6.5e-13 |                    1.5e-13
              1000.0 |       130 |                        7.1e-09 |                    1.6e-09


In [21]:
for i in range(len(rp2)):
    print("%30.2g"%(rp2[i]*sizeList[i]**2*pi))

                       1.1e-27
                       4.4e-23
                       4.7e-19
                         5e-11


In [22]:
for i in range(len(sizeList)):
    print("%30.2g"%(mStar*G*4/3*sizeList[i]**3*2/dist**2))

                         3e-17
                         3e-14
                         3e-11
                         3e-05


In [23]:
def betaprrp(sizeList, pr, rp, dist, mstar, density):
    l = len(sizeList)
    count = 0
    bpr = [0]*l
    brp = [0]*l
    while count < l:
        mdust = 4/3*pi*sizeList[count]**3*density
        fgrav = G*mstar*mdust/dist**2
        bpr[count] = pr[count]/fgrav
        brp[count] = rp[count]*4*pi*sizeList[count]**2/fgrav #convert to force
        count+=1
    return bpr, brp
    

In [24]:
betapr1, betarp1 = betaprrp(sizeList, pr1, rp1, dist, mStar, 2)
betapr2, betarp2 = betaprrp(sizeList, pr2, rp2, 130*AU, mStar, 2)


In [30]:
def timescale(betapr, betarp, dist, mStar, rStar):
    l = len(betapr)
    count = 0
    tscale = [0]*l
    while count < l:
        a_max = -G*mStar*(1-betarp[count]+betapr[count])/rStar**2
        a_ave = a_max/2
        tscale[count] = (2*dist/abs(a_ave))**(1/2)
        print(betarp[count],betapr[count])
        count+=1
        
    return tscale

In [32]:
def timescale2(sizeList, pr, rp, dist, mstar, rstar, density):
    l = len(sizeList)
    count = 0
    tscale = [0]*l
    while count < l:
        mdust = 4/3*pi*sizeList[count]**3*density
        tscale[count] = mdust/(pr[count]- rp[count])*(G*mstar*(1/rstar - 1/dist))**(1/2)
        count+=1
    return tscale
        

In [31]:
tscale1 = timescale(betapr1, betarp1, dist, mStar, rStar)
tscale2 = timescale(betapr2, betarp2, 130*AU, mStar, rStar)


7.551515022259152e-09 343.5719817510043
3.132796556041372e-07 142.5331364645622
3.2995781023633798e-06 15.012121199913306
0.0003562177983206022 0.16206874321674203
7.551514583294343e-09 26.428612444567808
3.132796392081147e-07 10.964086846526813
3.2995782768835715e-06 1.1547786149176502
0.00035621787934158125 0.01246682923684094


In [27]:
print(tscale1)
print(tscale2)

[10941.686626722847, 16953.062595237076, 50757.457891818834, 188440.79018266825]
[139827.9561865374, 211717.20696659418, 498878.66681861854, 727917.1299557134]


In [33]:
tscale11 = timescale2(sizeList, pr1, rp1, dist, mStar, rStar, 2)
tscale22 = timescale2(sizeList, pr2, rp2, 130*AU, mStar, rStar, 2)

In [36]:
aaa = len(tscale11)
bbb = 0
while bbb < aaa :
    print(tscale11[bbb]/(pi*1e7))
    print(tscale22[bbb]/(pi*1e7))
    print('-------------------------------------')
    bbb+=1

0.36254069785013704
1013.2921621020536
-------------------------------------
0.8738938124520436
2442.5112843571333
-------------------------------------
8.297216920050111
23190.51071726254
-------------------------------------
768.5555126393826
2148092.7777670682
-------------------------------------
